In [1]:
from fishsense_api_sdk.client import Client
from label_studio_sdk.client import LabelStudio
from pathlib import Path
import cv2
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime
from copy import deepcopy

In [2]:
URL = "http://localhost:8000"
DIVE_ID = 347
PROJECT_ID = 60

In [3]:
OUTPUT_FOLDER = (Path("../output") / "preprocess_groups_jpeg").absolute()

In [4]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [5]:
fs = Client(URL)
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [6]:
dive = await fs.dives.get(dive_id=DIVE_ID)

dive

Dive(id=347, name='031424_Alligator0_FSL02', path='2025-02-10 REEF Data Dump SMILE 6/031424_Alligator_FSL02/031424_Alligator0_FSL02', dive_datetime=datetime.datetime(2024, 3, 14, 12, 15, 46, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=2, dive_slate_id=None)

In [7]:
images = await fs.images.get(dive_id=dive.id)

len(images), images[0]

(333,
 Image(id=102731, path='2025-02-10 REEF Data Dump SMILE 6/031424_Alligator_FSL02/031424_Alligator0_FSL02/P3140042.ORF', taken_datetime=datetime.datetime(2024, 3, 14, 11, 19, 48, tzinfo=TzInfo(0)), checksum='56975f26d2e2c1281fda306fa95e7baf', is_canonical=True, dive_id=347, camera_id=2))

In [8]:
species_labels = pd.read_csv("../scripts/species_labels.csv", keep_default_na=False)

species_labels

,Index,Column 8,Checksum,File Name,Species,Measurable,Not Center Fish,Column 6,Comments/Notes,Fish Angles/Curved Tail
0,0.00,2,a5dbaee04184b279e21745b917f85786,0_a5dbaee04184b279e21745b917f85786.JPG,Grey Snapper (Lutjanus griseus),True,FALSE,,,
1,1.00,2,520d35c076e36092c243f5cf6f02a7dc,1_520d35c076e36092c243f5cf6f02a7dc.JPG,Other (Identifiable but Nontarget),True,FALSE,,,
2,2.00,2,cc97ac368241f5f582541fd29f141a6b,2_cc97ac368241f5f582541fd29f141a6b.JPG,Hogfish (Lachnolaimus maximus),True,FALSE,,I don't think this one should be labeled measu...,Significant Curve
3,3.00,2,0ea63f5f073b0c6c4749c86ddd666925,3_0ea63f5f073b0c6c4749c86ddd666925.JPG,Stoplight Parrotfish (Sparisoma viride),False,FALSE,,,
4,4.00,2,f18aebcf2cfe3a390c3407d05085d0de,4_f18aebcf2cfe3a390c3407d05085d0de.JPG,Grey Snapper (Lutjanus griseus),True,FALSE,,,
...,...,...,...,...,...,...,...,...,...,...
2995,"2,995.00",5,53b300ae61154e8bdf9c31284f4b7eed,2995_53b300ae61154e8bdf9c31284f4b7eed.JPG,Stoplight Parrotfish (Sparisoma viride),True,FALSE,,,
2996,"2,996.00",5,f144e1420f1b40db9027bf79d37f8f6c,2996_f144e1420f1b40db9027bf79d37f8f6c.JPG,Stoplight Parrotfish (Sparisoma viride),False,FALSE,,,
2997,"2,997.00",5,e91bb07fce97f762157f6c56da6cc745,2997_e91bb07fce97f762157f6c56da6cc745.JPG,Stoplight Parrotfish (Sparisoma viride),True,TRUE,,,
2998,"2,998.00",5,eecf2abc577233c625e889db2ccfca63,2998_eecf2abc577233c625e889db2ccfca63.JPG,Unidentifiable (Cannot see),False,TRUE,,,


In [9]:
image_checksums = {img.checksum for img in images}

len(image_checksums), list(image_checksums)[:5]

(333,
 ['d7be35187447a615b4eaaa9bd7f71d9e',
  '1a4530da77d15e775de7e6bf865171d7',
  '94e90150c469ae6570f1e2d64b725d4b',
  '945c46bfd8ab73b4e22e4162c5e6c8b8',
  'ac71c562074f20b521d5a438375f1f42'])

In [10]:
filtered_species_labels = {
    species_labels.iloc[idx]["Checksum"]:(species_labels.iloc[idx]["Species"], species_labels.iloc[idx]["Measurable"], species_labels.iloc[idx]["Not Center Fish"], species_labels.iloc[idx]["Column 6"], species_labels.iloc[idx]["Comments/Notes"], species_labels.iloc[idx]["Fish Angles/Curved Tail"])
    for idx in range(species_labels.shape[0])
    if species_labels.iloc[idx]["Checksum"] in image_checksums and species_labels.iloc[idx]["Species"] != 'Slate'
}

len(filtered_species_labels)

0

In [11]:
len([i for _, _, _, _, _, i in filtered_species_labels.values() if i])

0

In [12]:
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

now

'2025-10-26 12:56:21'

In [13]:
filtered_species_labels_list = list(l for _, l, _, _, _, _ in filtered_species_labels.values() if l)

len(filtered_species_labels_list)

0

In [14]:
fish_angle_options = {
    "x < 5°",
    "5° < x < 10°",
    "10° < x < 15°"
}

curved_tail_options = {
    "No Curve",
    "Slight Curve",
    "Significant Curve"
}

fish_angle_options, curved_tail_options

({'10° < x < 15°', '5° < x < 10°', 'x < 5°'},
 {'No Curve', 'Significant Curve', 'Slight Curve'})

In [15]:
tasks = []
template = {
    "model_version": f"sql_labels_{now}",
    "result": []
}

for image in tqdm(images):
    laser_label = await fs.labels.get_laser_label(image_id=image.id)
    species_label = None
    if image.checksum in filtered_species_labels:
        species_label = filtered_species_labels[image.checksum]
    image_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

    img = cv2.imread(str(image_path))
    height, width, _ = img.shape

    predictions = []
    if laser_label is not None and laser_label.label is not None:
        prediction = deepcopy(template)
        predictions = [prediction]

        prediction["result"].append({
            "id": "laser_result",
            "type": "keypointlabels",
            "to_name": "image", "from_name": "laser",
            "original_width": width, "original_height": height,
            "image_rotation": 0,
            "value": {
                "x": laser_label.x / width * 100,
                "y": laser_label.y / height * 100,
                "width": 0.2,
                "keypointlabels": [laser_label.label]
            }
        })

    if species_label is not None:
        if len(predictions) == 0:
            prediction = deepcopy(template)
            predictions = [prediction]
        else:
            prediction = predictions[0]

        species, measurable, not_center_fish, _, _, fish_angles_curved_tail = species_label

        # We won't use the slate labels since we didn't give options.
        prediction["result"].append({
            "id": "species_result",
            "type": "taxonomy",
            "to_name": "image", "from_name": "species",
            "value": {
                "taxonomy": [
                    [
                        "Fish",
                        species
                    ]
                ]
            }
        })

        measurable_value = "no"
        if not_center_fish:
            measurable_value = "yes, not center of fish"
        else:
            measurable_value = "yes, center of fish"

        prediction["result"].append({
            "id": "measurable_result",
            "type": "taxonomy",
            "to_name": "image", "from_name": "measurable",
            "value": {
                "taxonomy": [
                    [
                        measurable_value
                    ]
                ]
            }
        })

        for item in fish_angles_curved_tail.split(","):
            item = item.strip()

            if item in fish_angle_options:
                prediction["result"].append({
                    "id": "fish_angle_result",
                    "type": "taxonomy",
                    "to_name": "image", "from_name": "fishAngles",
                    "value": {
                        "taxonomy": [
                            [
                                item
                            ]
                        ]
                    }
                })
            elif item in curved_tail_options:
                prediction["result"].append({
                    "id": "curved_tail_result",
                    "type": "taxonomy",
                    "to_name": "image", "from_name": "fishCurve",
                    "value": {
                        "taxonomy": [
                            [
                                item
                            ]
                        ]
                    }
                })

    tasks.append({
        "data": {
            "image": f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/{image.checksum}"
        },
        "predictions": predictions,
        "annotations": []
    })

  0%|          | 0/333 [00:00<?, ?it/s]

In [16]:
tasks

[{'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/56975f26d2e2c1281fda306fa95e7baf'},
  'predictions': [{'model_version': 'sql_labels_2025-10-26 12:56:21',
    'result': [{'id': 'laser_result',
      'type': 'keypointlabels',
      'to_name': 'image',
      'from_name': 'laser',
      'original_width': 4014,
      'original_height': 3016,
      'image_rotation': 0,
      'value': {'x': 49.051615631150746,
       'y': 40.740401716063914,
       'width': 0.2,
       'keypointlabels': ['Green Laser']}}]}],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/39d87ebf5c0ff5421c17fa9e2f39a2b9'},
  'predictions': [{'model_version': 'sql_labels_2025-10-26 12:56:21',
    'result': [{'id': 'laser_result',
      'type': 'keypointlabels',
      'to_name': 'image',
      'from_name': 'laser',
      'original_width': 4014,
      'original_height': 3016,
      'image_rotation': 0,
      'value': {'x': 48

In [17]:
imported_tasks = ls.projects.import_tasks(PROJECT_ID, request=tasks, return_task_ids=True)

imported_tasks

ProjectsImportTasksResponse(annotation_count=0, could_be_tasks_list=False, data_columns=[], duration=0.411876916885376, file_upload_ids=[], found_formats=[], predictions_count=None, task_count=333, prediction_count=207, task_ids=[222963, 222964, 222965, 222966, 222967, 222968, 222969, 222970, 222971, 222972, 222973, 222974, 222975, 222976, 222977, 222978, 222979, 222980, 222981, 222982, 222983, 222984, 222985, 222986, 222987, 222988, 222989, 222990, 222991, 222992, 222993, 222994, 222995, 222996, 222997, 222998, 222999, 223000, 223001, 223002, 223003, 223004, 223005, 223006, 223007, 223008, 223009, 223010, 223011, 223012, 223013, 223014, 223015, 223016, 223017, 223018, 223019, 223020, 223021, 223022, 223023, 223024, 223025, 223026, 223027, 223028, 223029, 223030, 223031, 223032, 223033, 223034, 223035, 223036, 223037, 223038, 223039, 223040, 223041, 223042, 223043, 223044, 223045, 223046, 223047, 223048, 223049, 223050, 223051, 223052, 223053, 223054, 223055, 223056, 223057, 223058, 22